# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be passed as an image. Once a request is received, the API will convert load the image preprocess it and pass it to the model. There are two main functions in the API: init() and run(). The init() function loads the model and returns a scoring function. The run() function processes the images and uses the first function to score them.

    Note: Always make sure you don't have any lingering notebooks running (Shutdown previous notebooks). Otherwise it may cause GPU memory issue.

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from azureml.core import Workspace
from azureml.core.model import Model
from dotenv import set_key, find_dotenv
import logging
from testing_utilities import get_auth

In [4]:
env_path = find_dotenv(raise_error_if_not_found=True)

### Write and save driver script


In [105]:
%%writefile driver.py


import torch
import torchvision
from torchvision.models.detection.mask_rcnn import MaskRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import numpy as np
from pathlib import *
from azureml.contrib.services.aml_request import rawhttp
from azureml.core.model import Model
from toolz import compose
import numpy as np
import timeit as t
from PIL import Image, ImageOps
import logging


def _tensor_to_list(image_tensor):
    #return image_tensor.cpu().numpy()
    return image_tensor.tolist()

def _image_ref_to_pil_image(image_ref):
    """ Load image with PIL (RGB)
    """
    return Image.open(image_ref).convert("RGB")


def _pil_to_tensor(pil_image):
    img = np.array(pil_image, dtype = np.float32)  
    img_tensor = torchvision.transforms.functional.to_tensor(img)/255
    return img_tensor


def _create_scoring_func():
    """ Initialize MaskRCNN ResNet 50 Model
    """
    logger = logging.getLogger("model_driver")
    start = t.default_timer()
    model_name = "maskrcnn_resnet50_model"
    model_path = Model.get_model_path(model_name)
    checkpoint = model_path
    # Model class must be defined somewhere
    num_classes=91
    pretrained_backbone = False
    backbone = resnet_fpn_backbone('resnet50', pretrained_backbone)
    model = MaskRCNN(backbone, num_classes)

    #model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=False)
    #model.load_state_dict(torch.load('model_best.pth.tar')['state_dict'])
    #model = torch.load(model_path)
     
    state_dict = torch.load(checkpoint)
    model.load_state_dict(state_dict)
    
    
    model.eval()
    end = t.default_timer()

    loadTimeMsg = "Model loading time: {0} ms".format(round((end - start) * 1000, 2))
    logger.info(loadTimeMsg)

    def call_model(img_array_list):
        #img_array = np.stack(img_array_list)
        #img_array = preprocess_input(img_array)
       # preds = model.predict(img_array)
        
        with torch.no_grad():
            preds = model(img_array_list)
        #print(type(preds))
        #print(preds)
        for j in range(len(preds)):
            del preds[j]['masks'] 
        transformed_preds = []
        for i in range(len(preds)):            
            transformed_preds.append({
                key: _tensor_to_list(img_tensor) for key, img_tensor in preds[i].items()
            })
        return transformed_preds
               


    return call_model


def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = _create_scoring_func()

    def process_and_score(images_dict):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()
        logger.info("Scoring {} images".format(len(images_dict)))
        transform_input = compose(_pil_to_tensor, _image_ref_to_pil_image)
        transformed_dict = {
            key: transform_input(img_ref) for key, img_ref in images_dict.items()
        }
        #pdb.set_trace()
        dict_values_list = list(transformed_dict.values())
        preds = scoring_func(dict_values_list)
        #preds = dict(zip(transformed_dict.keys(), preds))
        end = t.default_timer()

        logger.info("Predictions: {0}".format(preds))
        logger.info("Predictions took {0} ms".format(round((end - start) * 1000, 2)))
        return (preds, "Computed in {0} ms".format(round((end - start) * 1000, 2)))

    return process_and_score


def init():
    """ Initialise the model and scoring function
    """
    global process_and_score
    process_and_score = get_model_api()


@rawhttp
def run(request):
    """ Make a prediction based on the data passed in using the preloaded model
    """
    return process_and_score(request.files)

Overwriting driver.py


### Test the Driver

In [110]:
logging.basicConfig(level=logging.DEBUG)

In [106]:
%run driver.py

In [ ]:
# Get workspace
# Load existing workspace from the config file info.

ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

In [7]:
model_path = Model.get_model_path("maskrcnn_resnet50_model", _workspace=ws)

In [8]:
model_path

'azureml-models/maskrcnn_resnet50_model/1/maskrcnn_resnet50.pth'

## Test the driver¶
We test the driver by passing data.

In [9]:
img_path = "./test_image.jpg"
print(Image.open(img_path).size)
img = Image.open(img_path)

(1920, 1080)


In [107]:
# Always make sure you don't have any lingering notebooks running. Otherwise it may cause GPU memory issue.
process_and_score = get_model_api()

In [108]:
resp = process_and_score({"test": open(img_path, "rb")})

In [109]:
resp

([{'boxes': [[678.3322143554688,
     680.0374145507812,
     784.1846923828125,
     993.3047485351562],
    [136.69444274902344, 633.5426635742188, 274.71533203125, 926.4990234375],
    [233.74720764160156,
     581.7813720703125,
     382.95440673828125,
     861.6013793945312],
    [1395.167236328125,
     440.8146057128906,
     1538.195556640625,
     639.5924682617188],
    [1422.93359375, 579.0057373046875, 1476.787353515625, 638.4216918945312],
    [699.564697265625, 731.795166015625, 745.90185546875, 826.1213989257812],
    [925.3984375, 340.40972900390625, 1026.978515625, 490.4237060546875],
    [1003.208740234375,
     397.6317443847656,
     1058.7530517578125,
     498.1560363769531],
    [529.6865844726562,
     275.9363098144531,
     913.7566528320312,
     472.7071533203125],
    [298.8797912597656,
     584.5956420898438,
     382.6814880371094,
     807.8283081054688],
    [933.8543090820312,
     439.1072082519531,
     1008.7671508789062,
     496.53955078125],
  

Next we will proceed with notebook [033_BuildImageForMLModule.ipynb](033_BuildImageForMLModule.ipynb).